In [1]:
import pandas as pd
import re
import spacy
from collections import Counter
import unicodedata
from bs4 import BeautifulSoup
from boilerpipe.extract import Extractor
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from nltk.corpus import stopwords
stopwords = set(stopwords.words())
INPUT_FILE = "../1-Data/3-annotation/output.csv"
# INPUT_FILE = "filtered_by_company_confidence.csv"

nlp = spacy.load('en_core_web_sm')
alternative_company_names = {"AMD (Advanced Micro Devices)": "AMD",
                    'Royal Dutch Shell PLC': "Shell",
                    "Samsung Electronics Co., Ltd.": "Samsung",
                    "Goodyear Tire & Rubber Co": "Goodyear",
                    "Sumitomo Rubber Industries": "Sumitomo",
                    "Exxon Mobil Corp.": "ExxonMobil",
                    "General Motors Corp.": "GM",
                    "Ford Motor Co.": "Ford",
                    "Toyota Motor Corp.": "Toyota",
                    "Petro China": "PetroChina",
                    'Volkswagen AG': "VW"}

def clean_text(html):
    soup = BeautifulSoup(html, "html.parser") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    texts = soup.findAll(text=True)
#     import ipdb; ipdb.set_trace()
#     print(len(texts))
    text = ". ".join(t.strip() for t in texts)
    text = unicodedata.normalize("NFKD", text)
    return text

Clean text from html tags

In [2]:
len(df)

404

In [3]:
df = pd.read_csv(INPUT_FILE)
df['text'] = df.apply(lambda row: "{} {}".format(row['title'], clean_text(str(row['content']))), axis=1)
df.drop(df[df.text.str.len() < 150].index, inplace=True)

In [4]:
len(df)

390

In [5]:
# COMPANY_NAMES_STOP_WORDS = "PLC|Corp"
with open("company-suffix.txt", "r") as fl:
    text = [i for i in fl.read().split('\n') if not i.startswith('//')]
    COMPANY_NAMES_STOP_WORDS = "(" + "$)|(".join(text) + "$)"

In [6]:
def get_company_names(company):
    company_names = [re.sub(COMPANY_NAMES_STOP_WORDS, '', company, flags=re.IGNORECASE).strip().lower()]
    if company in alternative_company_names:
        company_names.append(alternative_company_names[company].lower())
    #Company is often mentioned by part of it's name. e.g. "Royal Dutch Shell" -> "Shell"
#     company_names = set([company] + [i for i in company.split() if len(i)>2])
    return company_names

In [7]:
# pd.options.display.max_colwidth = 10000
# row = df.loc[df['url'] == 'http://www.sustainablebrands.com/solutionproviders/basf']
# row

## Sentiment Baseline:  
Using the same BoW technique as above  
I exclude texts that are not related to the company or climate change

In [8]:
target_names=('Strongly Negative', 'Negative', 'Neutral', 'Positive', 'Strongly Positive')
cleaned_df = df[(df['company_confidence'] != 0) & (df['climate_confidence'] != 0)]
cleaned_df = cleaned_df.dropna(subset =["sentiment"])

# cleaned_df.dropna(subset=['sentiment'], how='all', inplace = True)
simple_dc = {"Strongly Negative": '0', "Negative": '0', "Neutral": '2', "Positive": '1', "Strongly Positive": '1'}
cleaned_df['simple_sentiment'] = cleaned_df.apply(lambda row: simple_dc[row['sentiment']] ,axis=1)
print("Total: {} texts".format(len(cleaned_df)))
print("Sentiment counts:")
print(cleaned_df['sentiment'].value_counts(normalize=True))
print("Simple Sentiment counts:")
print(cleaned_df['simple_sentiment'].value_counts(normalize=True))

Total: 254 texts
Sentiment counts:
Positive             0.409449
Negative             0.220472
Strongly Positive    0.149606
Neutral              0.149606
Strongly Negative    0.070866
Name: sentiment, dtype: float64
Simple Sentiment counts:
1    0.559055
0    0.291339
2    0.149606
Name: simple_sentiment, dtype: float64


# BoW Again  

taken from https://gist.github.com/bbengfort/044682e76def583a12e6c09209c664a1

In [3]:
import os
import time
import string
import pickle

from operator import itemgetter

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split as tts


def timeit(func):
    """
    Simple timing decorator
    """
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = func(*args, **kwargs)
        delta  = time.time() - start
        return result, delta
    return wrapper


def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg


class NLTKPreprocessor(BaseEstimator, TransformerMixin):
    """
    Transforms input data by using NLTK tokenization, lemmatization, and
    other normalization and filtering techniques.
    """

    def __init__(self, stopwords=None, punct=None, lower=True, strip=True):
        """
        Instantiates the preprocessor, which make load corpora, models, or do
        other time-intenstive NLTK data loading.
        """
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = set(stopwords) if stopwords else set(sw.words('english'))
        self.punct      = set(punct) if punct else set(string.punctuation)
        self.punct.add("“")
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        """
        Fit simply returns self, no other information is needed.
        """
        return self

    def inverse_transform(self, X):
        """
        No inverse transformation
        """
        return X

    def transform(self, X):
        """
        Actually runs the preprocessing on each document.
        """
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):
        """
        Returns a normalized, lemmatized list of tokens from a document by
        applying segmentation (breaking into sentences), then word/punctuation
        tokenization, and finally part of speech tagging. It uses the part of
        speech tags to look up the lemma in WordNet, and returns the lowercase
        version of all the words, removing stopwords and punctuation.
        """
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                token = token.strip() if self.strip else token
                token = token.strip('_') if self.strip else token
                token = token.strip('*') if self.strip else token

                # If punctuation or stopword, ignore token and continue
                if token in self.stopwords or all(char in self.punct for char in token):
                    continue
                if tag == "NNP":
                    continue
                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        """
        Converts the Penn Treebank tag to a WordNet POS tag, then uses that
        tag to perform much more accurate WordNet lemmatization.
        """
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)



@timeit
def build_and_evaluate(X, y, classifier=SGDClassifier, outpath=None, verbose=True):
    """
    Builds a classifer for the given list of documents and targets in two
    stages: the first does a train/test split and prints a classifier report,
    the second rebuilds the model on the entire corpus and returns it for
    operationalization.
    X: a list or iterable of raw strings, each representing a document.
    y: a list or iterable of labels, which will be label encoded.
    Can specify the classifier to build with: if a class is specified then
    this will build the model with the Scikit-Learn defaults, if an instance
    is given, then it will be used directly in the build pipeline.
    If outpath is given, this function will write the model as a pickle.
    If verbose, this function will print out information to the command line.
    """

    @timeit
    def build(classifier, X, y=None):
        """
        Inner build function that builds a single model.
        """
        if isinstance(classifier, type):
            classifier = classifier(loss='log')

        model = Pipeline([
            ('preprocessor', NLTKPreprocessor()),
            ('vectorizer', TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False)),
            ('classifier', classifier),
        ])

        model.fit(X, y)
        return model

    # Label encode the targets
    labels = LabelEncoder()
    y = labels.fit_transform(y)

    # Begin evaluation
    if verbose: print("Building for evaluation")
    X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
    model, secs = build(classifier, X_train, y_train)

    if verbose: print("Evaluation model fit in {:0.3f} seconds".format(secs))
    if verbose: print("Classification Report:\n")

    y_pred = model.predict(X_test)
    print(clsr(y_test, y_pred, target_names=labels.classes_))

    if verbose: print("Building complete model and saving ...")
    model, secs = build(classifier, X, y)
    model.labels_ = labels

    if verbose: print("Complete model fit in {:0.3f} seconds".format(secs))

    if outpath:
        with open(outpath, 'wb') as f:
            pickle.dump(model, f)

        print("Model written out to {}".format(outpath))

    return model


def show_most_informative_features(model, text=None, n=20):
    """
    Accepts a Pipeline with a classifer and a TfidfVectorizer and computes
    the n most informative features of the model. If text is given, then will
    compute the most informative features for classifying that text.
    Note that this function will only work on linear models with coefs_
    """
    # Extract the vectorizer and the classifier from the pipeline
    vectorizer = model.named_steps['vectorizer']
    classifier = model.named_steps['classifier']

    # Check to make sure that we can perform this computation
    if not hasattr(classifier, 'coef_'):
        raise TypeError(
            "Cannot compute most informative features on {} model.".format(
                classifier.__class__.__name__
            )
        )

    if text is not None:
        # Compute the coefficients for the text
        tvec = model.transform([text]).toarray()
    else:
        # Otherwise simply use the coefficients
        tvec = classifier.coef_

    # Zip the feature names with the coefs and sort
    coefs = sorted(
        zip(tvec[0], vectorizer.get_feature_names()),
        key=itemgetter(0), reverse=False
    )

    topn  = zip(coefs[:n], coefs[:-(n+1):-1])

    # Create the output string to return
    output = []

    # If text, add the predicted value to the output.
    if text is not None:
        output.append("\"{}\"".format(text))
        output.append("Classified as: {}".format(model.predict([text])))
        output.append("")

    # Create two columns with most negative and most positive features.
    for (cp, fnp), (cn, fnn) in topn:
        output.append(
            "{:0.4f}{: >15}    {:0.4f}{: >15}".format(cp, fnp, cn, fnn)
        )

    return "\n".join(output)


/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# lemmatizer = WordNetLemmatizer()
# stopwords = set(sw.words('english'))
# def lemmatize(token, tag):
#     """
#     Converts the Penn Treebank tag to a WordNet POS tag, then uses that
#     tag to perform much more accurate WordNet lemmatization.
#     """
#     tag = {
#         'N': wn.NOUN,
#         'V': wn.VERB,
#         'R': wn.ADV,
#         'J': wn.ADJ
#     }.get(tag[0], wn.NOUN)

#     return lemmatizer.lemmatize(token, tag)

# def tokenize(document, company_names):
#     doc_tokens = []

#     for sent in sent_tokenize(document):
#         company_in_sent = False
#         sent_tokens = []
#         # Break the sentence into part of speech tagged tokens
#         for token, tag in pos_tag(wordpunct_tokenize(sent)):
#             # Apply preprocessing to the token
#             token = token.lower()
#             token = token.strip()
#             token = token.strip('_')
#             token = token.strip('*')

#             # If punctuation or stopword, ignore token and continue
# #             import ipdb; ipdb.set_trace()
#             if token in stopwords or all(char in set(string.punctuation) for char in token):
#                 continue
#             if tag == "NNP":
#                 continue
#             # Lemmatize the token and yield
#             lemma = lemmatize(token, tag)
#             if lemma in company_names:
#                 company_in_sent = True
#             sent_tokens.append(lemma)
#         if company_in_sent:
            
#             doc_tokens.extend(sent_tokens)
# #                 break
# #     if len(doc_tokens) == 0:
# #         print(document)
# #         print(company_names)
# #         import ipdb; ipdb.set_trace()
# #     print(company_names[0], len(doc_tokens))
#     return doc_tokens

In [11]:
get_company_names(list(cleaned_df["company"])[1])
# list(cleaned_df["company"])[1]

['chevron']

In [12]:
# get_company_names(list(cleaned_df["company"])[10])
# cleaned_df['lemmatized'] = cleaned_df.apply(lambda row: " ".join(tokenize(row['text'], get_company_names(row["company"]))), axis=1)

In [13]:
# list(cleaned_df["lemmatized"])[0]
# cleaned_df["sentiment"].unique()

array(['Positive', 'Negative', 'Neutral', 'Strongly Positive',
       'Strongly Negative'], dtype=object)

In [14]:
X = cleaned_df["text"]
y = cleaned_df["sentiment"]
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,
                                                    stratify=y)
model, secs = build_and_evaluate(X,y)
print(show_most_informative_features(model))

Building for evaluation


/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Evaluation model fit in 22.351 seconds
Classification Report:



/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                   precision    recall  f1-score   support

         Negative       0.75      0.86      0.80        14
          Neutral       0.38      0.33      0.35         9
         Positive       0.67      0.67      0.67        21
Strongly Negative       0.00      0.00      0.00         3
Strongly Positive       0.33      0.50      0.40         4

      avg / total       0.57      0.61      0.59        51

Building complete model and saving ...
Complete model fit in 31.939 seconds
-3.1685            use    5.0256          shale
-3.0243            cut    4.1701            sea
-2.7427         global    3.6310         diesel
-2.7054         energy    3.2205            ice
-2.4729        address    2.9910      fiduciary
-2.3766     technology    2.9726       refinery
-2.3696       electric    2.9132           file
-2.3612            ”..    2.9015       sanction
-2.2938           well    2.7197           coal
-2.2832         reduce    2.6124             lb
-2.2554       business    2.

/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [18]:
X = cleaned_df["text"]
y = cleaned_df["simple_sentiment"]
# test_size = 0.2
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42,
#                                                     stratify=y)
model, secs = build_and_evaluate(X,y, outpath="model.pickle")
print(show_most_informative_features(model))

Building for evaluation


/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Evaluation model fit in 26.622 seconds
Classification Report:

             precision    recall  f1-score   support

          0       0.61      0.85      0.71        13
          1       0.86      0.83      0.85        30
          2       0.50      0.25      0.33         8

avg / total       0.74      0.75      0.73        51

Building complete model and saving ...
Complete model fit in 29.723 seconds
Model written out to model.pickle
-3.2397         energy    5.0494          shale
-3.0996          water    4.8674       fracking
-2.9551        project    4.0966       sanction
-2.6066            use    3.4727            sea
-2.5935        address    3.1131            ban
-2.4788       football    3.0083        company
-2.4677     technology    2.9371         diesel
-2.3629       electric    2.9031            ice
-2.3218           list    2.8245            bee
-2.3118          power    2.7541     resolution
-2.2919        vehicle    2.5953       refinery
-2.2447         global    2.509

/Users/bmarchenko/projects/prj-nlp/env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


# Okay, let's estimate 50 companies

In [4]:
import pickle
with open("model.pickle", "rb") as fl:
    model = pickle.load(fl)
df = pd.read_csv("../3-Improvement/filtered_by_company_confidence.csv")
df.columns
df['text'] = df.apply(lambda row: "{} {}".format(row['title'], clean_text(str(row['content']))), axis=1)
df.drop(df[df.text.str.len() < 150].index, inplace=True)

In [4]:
len(df)

1734

In [72]:
companies = df.groupby("company", as_index=False)
rows_list = []
for num, cp in enumerate(list(companies.groups.keys())):
    neg_url = ''
    pos_url = ''
    cp_df = df.loc[df['company'] == cp]
#     cp_df['lemmatized'] = cp_df.apply(lambda row: " ".join(tokenize(row['text'], get_company_names(row[""]))), axis=1)
    urls = list(cp_df['url'])
#     cp_df['text'] = cp_df.apply(lambda row: "{} {}".format(row[''], clean_text(str(row['content']))), axis=1)
    
    res = model.predict(cp_df['text'])
    counter = Counter(res)
    probs = model.predict_proba(cp_df['text'])
    
    negs, pos, neut = zip(*probs)
    neg_url = urls[negs.index(max(negs))]
    pos_url = urls[pos.index(max(pos))]

    rows_list.append({"company": cp, "pos_c":Counter(res)[1], "neg_c": Counter(res)[0], 'pos_p': sum(pos), 'neg_p': sum(negs), 'neg_url': f"<a href='{neg_url}' target='_blank'>Clickme</a>", 'pos_url': f"<a href='{pos_url}' target='_blank'>Clickme</a>"})#, "neg_url": neg_url,"pos_url": pos_url })
    print(f"{cp}: p: {Counter(res)[1]}, n: {Counter(res)[0]},  'pos_p': {sum(pos)}, 'neg_p': {sum(negs)}, neg_url: {neg_url},  pos_url: {pos_url},")
    
final_df =  pd.DataFrame(rows_list) 

ABB: p: 39, n: 0,  'pos_p': 35.92405694592466, 'neg_p': 1.3782542741267276, neg_url: https://www.theguardian.com/travel/2012/may/13/st-abbs-scottish-borders-walk,  pos_url: https://www.coastline.fi/2016/2016/01/26/towards-sustainable-world/,
AMD (Advanced Micro Devices): p: 32, n: 0,  'pos_p': 24.191115901454413, 'neg_p': 4.365591100805477, neg_url: https://www.thenational.ae/business/markets/mubadala-investment-sells-shares-in-amd-1.616794,  pos_url: https://obamawhitehouse.archives.gov/the-press-office/2015/11/30/white-house-announces-additional-commitments-american-business-act,
Agnico-Eagle Mines Ltd.: p: 1, n: 0,  'pos_p': 0.6063837710774422, 'neg_p': 0.15435557514567547, neg_url: http://www.cannor.gc.ca/eng/1384801616560/1384801665094,  pos_url: http://www.cannor.gc.ca/eng/1384801616560/1384801665094,
Amazon.com Inc.: p: 7, n: 0,  'pos_p': 6.013115575552231, 'neg_p': 1.1223583856013903, neg_url: https://corpgov.law.harvard.edu/2016/09/06/climate-change-sustainability-and-other-en

Johnson & Johnson: p: 2, n: 0,  'pos_p': 1.3642093369180044, 'neg_p': 0.484530701659247, neg_url: https://www.worldwildlife.org/press-releases/catalyst-coca-cola-hp-johnson-johnson-and-national-geographic-announce-goals-to-combat-climate-change,  pos_url: https://www.greenbiz.com/article/johnson-johnson-loreal-step-deforestation,
Kimberly-Clark Corp.: p: 4, n: 0,  'pos_p': 3.0668429064059026, 'neg_p': 0.23664517350104386, neg_url: http://www.csrhub.com/CSR_and_sustainability_information/Kimberly-Clark-Corporation,  pos_url: https://hbr.org/2014/03/the-ceo-of-kimberly-clark-on-building-a-sustainable-company,
Michelin: p: 38, n: 0,  'pos_p': 30.602527780106364, 'neg_p': 5.01748293001539, neg_url: http://www.naharnet.com/stories/en/241949,  pos_url: https://www.engie.com/en/journalists/press-releases/engie-joins-michelin-in-investing-in-symbio-fcell/,
Microsoft Corp.: p: 25, n: 0,  'pos_p': 22.277117185035667, 'neg_p': 0.8546273799804804, neg_url: https://www.nbcnews.com/science/environme

In [82]:
categories = {"Food": ["Cargill Inc", "Nestle SA", "Procter & Gamble, Co.", "PepsiCo Inc.", "Bunge"],
            "Chemicals": ["BASF AG", "Johnson & Johnson", "Bayer AG", "Ineos", "Dow Chemical"],
            "Electronics": ["Samsung Electronics Co., Ltd.", "Sony Corp.", "AMD (Advanced Micro Devices)", "Toshiba Corp.", "Motorola Inc."],
            "Metals & mining": ["Glencore International AG", "ArcelorMittal", "Agnico-Eagle Mines Ltd.", "ThyssenKrupp Stahl", "Arcelor SA"],
            "Tires": ["Michelin", "Bridgestone Corp.", "Continental AG", "Goodyear Tire & Rubber Co", "Sumitomo Rubber Industries"],
            "Energy & water": ["GazProm", "Duke Energy", "Petrobras", "Petro China", "RosNeft", "BP plc", "Exxon Mobil Corp.", "Total SA", 'Royal Dutch Shell PLC', "Chevron Corp."],
            "Automotive & transport": ["General Motors Corp.", "BMW", "ABB", "Daimler AG", "Ford Motor Co.", "Toyota Motor Corp.", "Volkswagen AG"],
            "Wood & paper products": ["International Paper Corp.", "Georgia Pacific", "Weyerhaeuser Co.", "Stora Enso Oyj", "Kimberly-Clark Corp."],
            "Computer services & software": ["IBM", "Microsoft Corp.", "EDS Corp.", "AuFeminin.com", "Amazon.com Inc.", "Apple Inc"],
#             "Dirty Dozen": ["GazProm", "Petrobras", "RosNeft", "Petro China", "Duke Energy"]             
}
companies_dc = {}
for cat, companies in categories.items():
    for c in companies:
        companies_dc[c] = cat
cats = list(categories.keys())
final_df['category'] = final_df.apply(lambda row: companies_dc[row['company']], axis=1)
import matplotlib.pyplot as plt
from matplotlib import colors
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)
#http://www.climatechangenews.com/2015/05/21/the-dirty-dozen-the-fossil-fuel-industrys-polluting-league-table/
dirty_dozen = ["GazProm", "Petrobras", "RosNeft", "Petro China", "Glencore International AG", 'Royal Dutch Shell PLC', "Exxon Mobil Corp.", "BP plc", "Chevron Corp."]

def highlight_categories(row):
    return [f'background-color: {colors.rgb2hex(cmap(cats.index(row["category"])))}' for i in row]

def highlight_dirty_dozen(row):
    if row["company"] in dirty_dozen:
        return ['background-color: red' for i in row]
    else:
        return ["" for i in row]


### Order by absolute number of negatives/positives

In [80]:
from IPython.display import HTML
final_df['mark'] = final_df['neg_c']/final_df['pos_c']
final_df = final_df.sort_values(['mark'], ascending=[0])
# final_df['pos_url'] = final_df.apply(lambda row: f"<a href='{row['pos_url']}'>{row['pos_url']}</a>", axis=1)

final_df.style.apply(highlight_categories, axis=1)
# HTML(final_df.to_html(escape=False))
# final_df.style.apply(highlight_dirty_dozen, axis=1)

,company,neg_c,neg_p,neg_url,pos_c,pos_p,pos_url,category,mark
20,Exxon Mobil Corp.,37,31.1944,Clickme,8,10.9114,Clickme,Energy & water,4.625
27,Ineos,36,33.9565,Clickme,8,8.49772,Clickme,Chemicals,4.5
14,Chevron Corp.,81,71.5421,Clickme,19,24.2073,Clickme,Energy & water,4.26316
38,RosNeft,45,37.0411,Clickme,18,21.7179,Clickme,Energy & water,2.5
17,Dow Chemical,9,6.12734,Clickme,5,6.06841,Clickme,Chemicals,1.8
9,BP plc,16,13.7918,Clickme,10,9.20712,Clickme,Energy & water,1.6
39,Royal Dutch Shell PLC,37,35.0716,Clickme,41,39.075,Clickme,Energy & water,0.902439
36,Petro China,11,9.85226,Clickme,13,12.0754,Clickme,Energy & water,0.846154
46,Volkswagen AG,14,12.3594,Clickme,17,16.4614,Clickme,Automotive & transport,0.823529
37,Petrobras,33,32.2488,Clickme,58,51.7571,Clickme,Energy & water,0.568966


# Naive validation: highlight Dirty Dozen

In [83]:
final_df.style.apply(highlight_dirty_dozen, axis=1)

,company,neg_c,neg_p,neg_url,pos_c,pos_p,pos_url,category,mark
20,Exxon Mobil Corp.,37,31.1944,Clickme,8,10.9114,Clickme,Energy & water,4.625
27,Ineos,36,33.9565,Clickme,8,8.49772,Clickme,Chemicals,4.5
14,Chevron Corp.,81,71.5421,Clickme,19,24.2073,Clickme,Energy & water,4.26316
38,RosNeft,45,37.0411,Clickme,18,21.7179,Clickme,Energy & water,2.5
17,Dow Chemical,9,6.12734,Clickme,5,6.06841,Clickme,Chemicals,1.8
9,BP plc,16,13.7918,Clickme,10,9.20712,Clickme,Energy & water,1.6
39,Royal Dutch Shell PLC,37,35.0716,Clickme,41,39.075,Clickme,Energy & water,0.902439
36,Petro China,11,9.85226,Clickme,13,12.0754,Clickme,Energy & water,0.846154
46,Volkswagen AG,14,12.3594,Clickme,17,16.4614,Clickme,Automotive & transport,0.823529
37,Petrobras,33,32.2488,Clickme,58,51.7571,Clickme,Energy & water,0.568966


### Order by sum of probabilities negatives/positives

In [76]:
# final_df['mark'] = final_df['neg_p']/final_df['pos_p']
# final_df = final_df.sort_values(['mark'], ascending=[0])
# final_df.style.apply(highlight_categories, axis=1)

### Order by avg probability negatives/positives

In [77]:
# final_df['mark'] =  (final_df['pos_p']/final_df['pos_c'])- (final_df['neg_p']/final_df['neg_c'])
# final_df.sort_values(['mark'], ascending=[0])
# final_df.style.apply(highlight_categories, axis=1)

In [ ]:
# print(f"length before: {len(df)}")
# df['title'] = df['']
# df['company'] = df['']
# df['text'] = df.apply(lambda row: "{} {}".format(row[''], clean_text(str(row['content']))), axis=1)

# for num, row in df.iterrows():
#     cc = find_company_confidence(row)
#     if cc == 0:
        

In [ ]:
# df['company_confidence'] = df.apply(lambda row: find_company_confidence(row) ,axis=1)
# df = df[df['company_confidence'][0] != 0]
# print(f"length after: {len(df)}")

In [ ]:
# df['company_confidence'] = df.apply(lambda row: row['company_confidence'][0] ,axis=1)

In [ ]:
# df = df[df['company_confidence'] != 0]
# print(f"length after: {len(df)}")

In [ ]:
# companies = df.groupby("", as_index=False)
# rows_list = []
# for num, cp in enumerate(list(companies.groups.keys())):
#     cp_df = df.loc[df[''] == cp]
#     res = model.predict(cp_df['text'])
#     rows_list.append({"company": cp, "pos":Counter(res)[1], "neg": Counter(res)[0], "mark": Counter(res)[0]/Counter(res)[1] })
#     print(f"{cp}: p: {Counter(res)[1]}, n: {Counter(res)[0]}, {round(Counter(res)[0]/Counter(res)[1],3)}")
    
# final_df =  pd.DataFrame(rows_list) 
# final_df.sort_values(['mark'], ascending=[0])